<h1 style="background-color:red;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת שכבת חלקות (דו מימדיות) מבוטלות לבנק"ל מודרני 🌐

In [ ]:
print('19. Canceled parcels borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  19.a Reading inputs')

<h1 style="text-align:right;font-size:125%">
נתוני שכבת חלקות היסטוריות

In [ ]:
history_parcels_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'LEGAL_AREA', 'TALAR_NUM', 'TALAR_YEAR', 'geometry']

history_parcels = daskgpd.read_file(CNFG.CADSDE + 'PARCEL_HISTORY_HASAVA.gdb', layer = 'PARCEL_HISTORY_HASAVA', columns = history_parcels_cols, npartitions = 20).compute()

del history_parcels_cols

print('         input 1/7 read')

<h1 style="text-align:right;font-size:125%">
טבלת חלקות מבוטלות

In [ ]:
cancel_parcel_cols = ['F_GUSH_NUM', 'F_GUSH_SUFFIX', 'F_PARCEL_NUM', 'TALAR_NUMBER', 'TALAR_YEAR']

cancel_parcel = pd.read_csv(CNFG.Oracle + 'CANCEL_PARCEL.csv', usecols = cancel_parcel_cols)

del cancel_parcel_cols

print('         input 2/7 read')

<h1 style="text-align:right;font-size:125%">
שכבת חלקות מבוטלות מרפליקציות יומיות

In [ ]:
replicated_parcels_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'LEGALAREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

replicated_parcels = daskgpd.read_file(CNFG.Local + 'CanceldParcels.gdb',  layer = 'CanceledParcels', columns = replicated_parcels_cols, npartitions = 20).compute()

del replicated_parcels_cols

print('         input 3/7 read')

<h1 style="text-align:right;font-size:125%">
שכבת חלקות מבוטלות מרפליקציות יומיות

In [ ]:
sequence_cols = ['CPBUniqueID', 'ProcessType',  'BlockNumber', 'SubBlockNumber', 'FromParcelTemp', 'FromParcelFinal', 'ToBlockNumber', 'ToSubBlockNumber', 'ToParcelTemp', 'ToParcelFinal']

SequenceActions = pd.read_csv(CNFG.CSVPath + 'SequenceActions.csv', usecols = sequence_cols)

del sequence_cols

print('         input 4/7 read')

<h1 style="text-align:right;font-size:125%">
נתוני גושים מודרניים 

In [ ]:
blocks_cols = ['BlockUniqueID', 'Name', 'LandType']

Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = blocks_cols)
del blocks_cols

print('         input 5/7 read')

<h1 style="text-align:right;font-size:125%">
(פיצול לתהליכים יוצרים ותהליכים מעדכנים) נתוני שכבת גבולות תהליכי קדסטר

In [ ]:
cpb_cols = ['CPBUniqueID', 'GlobalID', 'ProcessName', 'ProcessType']

CancelingProcessTypes = [1, 2, 3, 11, 16] # תצ"ר, תמ"ר, פס"ד, תצ"ר לא מוסדר, עריכה חופשית

cpb_layer = daskgpd.read_file(CNFG.ProductPath, layer = 'CadasterProcessBorders', columns = cpb_cols, npartitions = 10).compute()

CancelingProcessLayer = cpb_layer[cpb_layer['ProcessType'].isin(CancelingProcessTypes)]

del [cpb_cols, cpb_layer]


print('         input 6/7 read')

<h1 style="text-align:right;font-size:125%">
שכבת חלקות דו-ממדיות מודרנית

In [ ]:
Parcels2D_List = pd.read_csv(CNFG.CSVPath + 'Parcels2DList.csv')

print('         input 7/7 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  15.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
עיבוד טבלת סדר פעולות לקבלת חלקות מתבטלות בלבד

In [ ]:
SequenceActions = SequenceActions[SequenceActions['ProcessType'].isin(CancelingProcessTypes)]

SequenceActions = SequenceActions[~SequenceActions['ToParcelFinal'].isna()]

SequenceActions['IsCanceled'] = np.where(SequenceActions['FromParcelFinal'] != SequenceActions['ToParcelFinal'], True, False)

SequenceActions = SequenceActions[SequenceActions['IsCanceled'] == True]

SequenceActions = SequenceActions[['FromParcelFinal', 'BlockNumber', 'SubBlockNumber', 'CPBUniqueID']]

SequenceActions = SequenceActions[(~SequenceActions['FromParcelFinal'].isna()) & (SequenceActions['FromParcelFinal'] != 0)]

SequenceActions.rename(columns = {'FromParcelFinal' : 'ParcelNumber'}, inplace = True)


del CancelingProcessTypes

<h1 style="text-align:right;font-size:100%">
עיבוד טבלת חלקות מבוטלות

In [ ]:
cancel_parcel.rename(columns = {'F_GUSH_NUM'    : 'BlockNumber',
                                'F_GUSH_SUFFIX' : 'SubBlockNumber',
                                'F_PARCEL_NUM'  : 'ParcelNumber'}, inplace = True)

cancel_parcel = cancel_parcel[(~cancel_parcel['TALAR_NUMBER'].isna())
                                              &
                              (~cancel_parcel['TALAR_YEAR'].isna())]


cancel_parcel[['TALAR_NUMBER', 'TALAR_YEAR']] = cancel_parcel[['TALAR_NUMBER', 'TALAR_YEAR']].astype(int)
cancel_parcel['ProcessName'] = cancel_parcel['TALAR_NUMBER'].astype(str) + '/' + cancel_parcel['TALAR_YEAR'].astype(str)
cancel_parcel.drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR'], inplace = True)

cancel_parcel = cancel_parcel.merge(CancelingProcessLayer, on = 'ProcessName', how = 'left')
cancel_parcel['CPBUniqueID'] = cancel_parcel['CPBUniqueID'].fillna(CNFG.unknown_cancel_process_id)

<h1 style="text-align:right;font-size:100%">
עיבוד נתוני חלקות מבוטלות מרפליקציות יומיות

In [ ]:
replicated_parcels.rename(columns = {'PARCEL'       : 'ParcelNumber',
                                     'GUSH_NUM'     : 'BlockNumber',
                                     'GUSH_SUFFIX'  : 'SubBlockNumber'}, inplace = True)

replicated_parcels = replicated_parcels[(~replicated_parcels['TALAR_NUMBER'].isna())
                                              &
                                        (~replicated_parcels['TALAR_YEAR'].isna())]

replicated_parcels[['TALAR_NUMBER', 'TALAR_YEAR']] = replicated_parcels[['TALAR_NUMBER', 'TALAR_YEAR']].astype(int)
replicated_parcels['ProcessName'] = replicated_parcels['TALAR_NUMBER'].astype(str) + '/' + replicated_parcels['TALAR_YEAR'].astype(str)
replicated_parcels.drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR'], inplace = True)

replicated_parcels = replicated_parcels.merge(CancelingProcessLayer, on = 'ProcessName', how = 'left')
replicated_parcels['CPBUniqueID'] = replicated_parcels['CPBUniqueID'].fillna(CNFG.unknown_cancel_process_id)

<h1 style="text-align:right;font-size:100%">
עיבוד נתוני חלקות היסטוריות

In [ ]:
history_parcels.rename(columns = {'GUSH_NUM'     : 'BlockNumber',
                                  'GUSH_SUFFIX'  : 'SubBlockNumber',
                                  'PARCEL'       : 'ParcelNumber',
                                  'LEGAL_AREA'   : 'StatedArea'}, inplace = True)

history_parcels = history_parcels[(~history_parcels['TALAR_NUM'].isna())
                                              &
                                (~history_parcels['TALAR_YEAR'].isna())]

history_parcels[['TALAR_NUM', 'TALAR_YEAR']] = history_parcels[['TALAR_NUM', 'TALAR_YEAR']].astype(int)
history_parcels['ProcessName'] = history_parcels['TALAR_NUM'].astype(str) + '/' + history_parcels['TALAR_YEAR'].astype(str)
history_parcels.drop(columns = ['TALAR_NUM', 'TALAR_YEAR'], inplace = True)

history_parcels = history_parcels.merge(CancelingProcessLayer, on = 'ProcessName', how = 'left')
history_parcels['CPBUniqueID'] = history_parcels['CPBUniqueID'].fillna(CNFG.unknown_cancel_process_id)

<h1 style="text-align:right;font-size:100%">
איחוד מקורות מידע של חלקות מבוטלות לטבלה אחת

In [ ]:
CancelParcels2D = pd.concat([cancel_parcel, replicated_parcels, history_parcels, SequenceActions])

CancelParcels2D['SubBlockNumber'] = CancelParcels2D['SubBlockNumber'].fillna(0)

CancelParcels2D = CancelParcels2D[(CancelParcels2D['ParcelNumber'] > 0)
                                                  &
                                  (~CancelParcels2D['ParcelNumber'].isna())]

del [CancelingProcessLayer, cancel_parcel, replicated_parcels, history_parcels, SequenceActions]

In [ ]:
aggregation = {'CPBUniqueID' : 'max',
               'GlobalID'    : 'first',
               'geometry'    : 'first',
               'StatedArea'  : 'max'}

CancelParcels2D = CancelParcels2D.groupby(by = ['ParcelNumber', 'BlockNumber', 'SubBlockNumber'], as_index = False).agg(aggregation)

In [ ]:
CancelParcels2D.rename(columns = {'CPBUniqueID' : 'CPBUniqueCancelProcessID', 'GlobalID' : 'RetiredByRecord', 'ProcessType' : 'CancelProcessType'}, inplace = True)

CancelProcessTypeMap = {1 : 1,       # תצ"ר
                        2 : 4,       # תמ"ר
                        3 : 2,       # פס"ד
                        11 :3,       # תצ"ר לא מוסדר
                        16 : None}   # עריכה חופשית

CancelParcels2D['CancelProcessType'] = CancelParcels2D['CancelProcessType'].map(CancelProcessTypeMap)

del aggregation

In [ ]:
CancelParcels2D[['BlockNumber', 'SubBlockNumber']] = CancelParcels2D[['BlockNumber', 'SubBlockNumber']].astype(int)

CancelParcels2D['BlockName'] = CancelParcels2D['BlockNumber'].astype(str) + '/' +  CancelParcels2D['SubBlockNumber'].astype(str) 

CancelParcels2D = CancelParcels2D.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')

del Blocks

In [ ]:
CancelParcels2D['ParcelNumber'] = CancelParcels2D['ParcelNumber'].astype(int)

CancelParcels2D['Name'] = CancelParcels2D['ParcelNumber'].astype(str) + CancelParcels2D['BlockName']

CancelParcels2D.drop(columns = 'BlockName')

In [ ]:
Parcels2D_List = Parcels2D_List.astype(int)
Parcels2D_List['Name'] = Parcels2D_List['ParcelNumber'].astype(str) + Parcels2D_List['BlockNumber'] + Parcels2D_List['SubBlockNumber'].astype(str)

CancelParcels2D = CancelParcels2D[~CancelParcels2D['Name'].isin(Parcels2D_List['Name'])]

In [ ]:
CancelParcels2D['ParcelType'] = 2 # סופית

In [ ]:
CancelParcels2D['StatedAreaUnit'] = 109404

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  19.e Exporting results to product folder')

In [ ]:
CancelParcels2D_no_geom = CancelParcels2D[CancelParcels2D['geometry'].isna()].drop(columns = 'geometry')
CancelParcels2D_geom = CancelParcels2D[~CancelParcels2D['geometry'].isna()]

In [ ]:
csv_dir = CNFG.CSVPath + 'CancelParcels2D_no_geom.csv'
json_dir = CNFG.JsonPath + 'CancelParcels2D.geojson'


CancelParcels2D_no_geom.to_csv(csv_dir, index = False)
gpd.GeoDataFrame(CancelParcels2D_geom, geometry = 'geometry', crs = 2039).to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + "CancelParcels2D-Save as Feature Class.bat"]);